## Imports

In [205]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from catboost.utils import select_threshold
import os
import catboost
from catboost import CatBoostClassifier
from catboost import Pool

## Dataset

In [206]:
Billing = pd.read_excel("./Customer_Billing.xlsx", index_col=0)
Demographics = pd.read_excel("./Customer_Usage.xlsx", index_col=0) 
Usage = pd.read_excel("./Customer_Demographics.xlsx", index_col=0) 

In [207]:
result = pd.merge(Billing, Demographics, how="inner", on="customerID")

In [208]:
result_final = pd.merge(result, Usage, how="inner", on="customerID")
result_final=result_final.set_index('customerID')
#result_final

## Data pre-processing

In [209]:
df = result_final
######
df = pd.get_dummies(df, prefix=['Contract_', 'PaperlessBilling_'], columns=['Contract', 'PaperlessBilling'])
df = df.drop(['Contract__Two year', 'PaperlessBilling__No'], axis=1)

######
#merge payment methods credit card and bank transfer to automatic
df = pd.get_dummies(df, prefix=['PaymentMethod_', 'gender_','SeniorCitizen_','Partner_','PhoneService_'], 
                    columns=['PaymentMethod', 'gender','SeniorCitizen','Partner','PhoneService'])
df = df.drop(['gender__Male', 'SeniorCitizen__0','Partner__No','PhoneService__No',
              'PaymentMethod__Bank transfer (automatic)','PaymentMethod__Credit card (automatic)'], axis=1)

######
df = pd.get_dummies(df, prefix=['Dependents_', 'Mul_tipleLines_','OnlineSecurity_','OnlineBackup_','DeviceProtection_'], 
                    columns=['Dependents', 'MultipleLines','OnlineSecurity','OnlineBackup','DeviceProtection'])
df = df.drop(['Dependents__No','Mul_tipleLines__No phone service','OnlineSecurity__No internet service',
              'OnlineBackup__No internet service','DeviceProtection__No internet service'], axis=1)

#####
df = pd.get_dummies(df, prefix=['InternetService_', 'TechSupport_','StreamingTV_','StreamingMovies_'], 
                    columns=['InternetService', 'TechSupport','StreamingTV','StreamingMovies'])
df = df.drop(['InternetService__No','TechSupport__No internet service',
              'StreamingTV__No internet service','StreamingMovies__No internet service'], axis=1)

#Chrun value
df = pd.get_dummies(df,prefix=['Churn_'],columns=['Churn'])
df = df.drop(['Churn__No'],axis=1)

#############################################################
# based on feature importance
df = df.drop(['gender__Female','Mul_tipleLines__No','Mul_tipleLines__Yes','PhoneService__Yes'],axis=1)


In [210]:
pd.set_option('display.max_columns', None)
df

,MonthlyCharges,TotalCharges,tenure,Contract__Month-to-month,Contract__One year,PaperlessBilling__Yes,PaymentMethod__Electronic check,PaymentMethod__Mailed check,SeniorCitizen__1,Partner__Yes,Dependents__Yes,OnlineSecurity__No,OnlineSecurity__Yes,OnlineBackup__No,OnlineBackup__Yes,DeviceProtection__No,DeviceProtection__Yes,InternetService__DSL,InternetService__Fiber optic,TechSupport__No,TechSupport__Yes,StreamingTV__No,StreamingTV__Yes,StreamingMovies__No,StreamingMovies__Yes,Churn__Yes
customerID,,,,,,,,,,,,,,,,,,,,,,,,,,
7590-VHVEG,29.85,29.85,1,1,0,1,1,0,0,1,0,1,0,0,1,1,0,1,0,1,0,1,0,1,0,0
5575-GNVDE,56.95,1889.5,34,0,1,0,0,1,0,0,0,0,1,1,0,0,1,1,0,1,0,1,0,1,0,0
3668-QPYBK,53.85,108.15,2,1,0,1,0,1,0,0,0,0,1,0,1,1,0,1,0,1,0,1,0,1,0,1
7795-CFOCW,42.30,1840.75,45,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,1,1,0,1,0,0
9237-HQITU,70.70,151.65,2,1,0,1,1,0,0,0,0,1,0,1,0,1,0,0,1,1,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6840-RESVB,84.80,1990.5,24,0,1,1,0,1,0,1,1,0,1,1,0,0,1,1,0,0,1,0,1,0,1,0
2234-XADUH,103.20,7362.9,72,0,1,1,0,0,0,1,1,1,0,0,1,0,1,0,1,1,0,0,1,0,1,0
4801-JZAZL,29.60,346.45,11,1,0,1,1,0,0,1,1,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0


In [211]:
#Change dtypes to float

#df['MonthlyCharges'] = df['MonthlyCharges'].str.replace(',','.')
df = df.astype({'MonthlyCharges':'float'})

df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'].astype('float')

#duplicateRows = df[df.duplicated(keep=False)]
#duplicateRows
#df.to_csv('df.csv')

df.dtypes

MonthlyCharges                     float64
TotalCharges                       float64
tenure                               int64
Contract__Month-to-month             uint8
Contract__One year                   uint8
PaperlessBilling__Yes                uint8
PaymentMethod__Electronic check      uint8
PaymentMethod__Mailed check          uint8
SeniorCitizen__1                     uint8
Partner__Yes                         uint8
Dependents__Yes                      uint8
OnlineSecurity__No                   uint8
OnlineSecurity__Yes                  uint8
OnlineBackup__No                     uint8
OnlineBackup__Yes                    uint8
DeviceProtection__No                 uint8
DeviceProtection__Yes                uint8
InternetService__DSL                 uint8
InternetService__Fiber optic         uint8
TechSupport__No                      uint8
TechSupport__Yes                     uint8
StreamingTV__No                      uint8
StreamingTV__Yes                     uint8
StreamingMo

## Train Test Split

In [212]:
### Create X and Y values
X = df.drop('Churn__Yes', axis=1)

#Y=result_final['Churn']
Y = df.loc[:, ['Churn__Yes']]

In [213]:
#0-2 features are not categorical
cat_features = list(range(3, X.shape[1]))
print(cat_features)

[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


In [214]:
### Split Train and Test Sets
X_TRAIN, X_test, y_TRAIN, y_test = train_test_split(X, Y, stratify=Y, test_size=0.2, random_state=24)

# XGBoost - CatBoost - CatBoost_weight 

In [215]:
from xgboost import XGBClassifier
#XGBoost
model_xgb = XGBClassifier(
    n_estimators=6000,
    learning_rate=0.0005,
    max_depth=10,
    #scale_pos_weight=4,
    #reg_lambda=5,
    #min_child_weight=1,
    #subsample=1,
    #colsample_bytree=1,
    eval_metric=['auc', 'error'],
    objective='binary:logistic'
)

In [216]:
#CatBoost
model_cat = CatBoostClassifier(
    iterations=6000,
    learning_rate=0.0005,
    depth=10,
    #scale_pos_weight=4,
    #l2_leaf_reg=5,
    #random_strength=1,
    #bagging_temperature=1
    #custom_loss=['AUC', 'Accuracy'],
    cat_features=cat_features
)

In [217]:
#CatBoost_weight
model_cat2 = CatBoostClassifier(
    iterations=6000,
    learning_rate=0.0005,
    depth=10,
    scale_pos_weight=4,
    #l2_leaf_reg=5,
    #random_strength=1,
    #bagging_temperature=1
    #custom_loss=['AUC', 'Accuracy'],
    cat_features=cat_features
)

# Voting Classifier

In [224]:
#Voting Classifier
from sklearn.ensemble import VotingClassifier
     
# Create the ensemble model
ensemble = VotingClassifier(estimators=[('Catboost2', model_cat2),('xgboost', model_xgb),('catboost', model_cat)], voting='soft')

# Fit the ensemble model on the training data
ensemble.fit(X_TRAIN, y_TRAIN)

# Get predictions on the test data
y_pred_ensemble = ensemble.predict(X_test)

import warnings
warnings.filterwarnings('ignore')

0:	learn: 0.6928672	total: 18.8ms	remaining: 1m 52s
1:	learn: 0.6925887	total: 39.3ms	remaining: 1m 57s
2:	learn: 0.6923099	total: 59.1ms	remaining: 1m 58s
3:	learn: 0.6920717	total: 79.1ms	remaining: 1m 58s
4:	learn: 0.6918014	total: 101ms	remaining: 2m
5:	learn: 0.6915073	total: 121ms	remaining: 2m
6:	learn: 0.6912794	total: 130ms	remaining: 1m 51s
7:	learn: 0.6910010	total: 148ms	remaining: 1m 50s
8:	learn: 0.6907231	total: 166ms	remaining: 1m 50s
9:	learn: 0.6904360	total: 184ms	remaining: 1m 50s
10:	learn: 0.6901691	total: 203ms	remaining: 1m 50s
11:	learn: 0.6898920	total: 221ms	remaining: 1m 50s
12:	learn: 0.6896165	total: 240ms	remaining: 1m 50s
13:	learn: 0.6893453	total: 262ms	remaining: 1m 51s
14:	learn: 0.6890784	total: 281ms	remaining: 1m 52s
15:	learn: 0.6888599	total: 286ms	remaining: 1m 46s
16:	learn: 0.6885946	total: 303ms	remaining: 1m 46s
17:	learn: 0.6883433	total: 321ms	remaining: 1m 46s
18:	learn: 0.6880772	total: 340ms	remaining: 1m 46s
19:	learn: 0.6878144	total

159:	learn: 0.6539625	total: 2.76s	remaining: 1m 40s
160:	learn: 0.6537248	total: 2.78s	remaining: 1m 40s
161:	learn: 0.6535030	total: 2.8s	remaining: 1m 40s
162:	learn: 0.6532502	total: 2.82s	remaining: 1m 40s
163:	learn: 0.6530144	total: 2.83s	remaining: 1m 40s
164:	learn: 0.6527911	total: 2.85s	remaining: 1m 40s
165:	learn: 0.6525782	total: 2.86s	remaining: 1m 40s
166:	learn: 0.6523554	total: 2.87s	remaining: 1m 40s
167:	learn: 0.6521441	total: 2.89s	remaining: 1m 40s
168:	learn: 0.6519518	total: 2.9s	remaining: 1m 39s
169:	learn: 0.6517459	total: 2.9s	remaining: 1m 39s
170:	learn: 0.6515542	total: 2.91s	remaining: 1m 39s
171:	learn: 0.6513085	total: 2.92s	remaining: 1m 39s
172:	learn: 0.6510909	total: 2.93s	remaining: 1m 38s
173:	learn: 0.6509243	total: 2.94s	remaining: 1m 38s
174:	learn: 0.6508075	total: 2.94s	remaining: 1m 37s
175:	learn: 0.6505880	total: 2.96s	remaining: 1m 37s
176:	learn: 0.6504194	total: 2.96s	remaining: 1m 37s
177:	learn: 0.6501793	total: 2.98s	remaining: 1m 

319:	learn: 0.6209320	total: 5.31s	remaining: 1m 34s
320:	learn: 0.6207337	total: 5.33s	remaining: 1m 34s
321:	learn: 0.6205240	total: 5.35s	remaining: 1m 34s
322:	learn: 0.6203423	total: 5.36s	remaining: 1m 34s
323:	learn: 0.6201778	total: 5.37s	remaining: 1m 34s
324:	learn: 0.6199758	total: 5.39s	remaining: 1m 34s
325:	learn: 0.6197745	total: 5.41s	remaining: 1m 34s
326:	learn: 0.6195735	total: 5.43s	remaining: 1m 34s
327:	learn: 0.6193799	total: 5.45s	remaining: 1m 34s
328:	learn: 0.6191817	total: 5.47s	remaining: 1m 34s
329:	learn: 0.6189782	total: 5.49s	remaining: 1m 34s
330:	learn: 0.6187835	total: 5.51s	remaining: 1m 34s
331:	learn: 0.6185849	total: 5.53s	remaining: 1m 34s
332:	learn: 0.6183999	total: 5.55s	remaining: 1m 34s
333:	learn: 0.6181943	total: 5.57s	remaining: 1m 34s
334:	learn: 0.6179917	total: 5.59s	remaining: 1m 34s
335:	learn: 0.6177713	total: 5.61s	remaining: 1m 34s
336:	learn: 0.6175627	total: 5.62s	remaining: 1m 34s
337:	learn: 0.6173573	total: 5.64s	remaining: 

483:	learn: 0.5916960	total: 8.12s	remaining: 1m 32s
484:	learn: 0.5915304	total: 8.14s	remaining: 1m 32s
485:	learn: 0.5913796	total: 8.14s	remaining: 1m 32s
486:	learn: 0.5912185	total: 8.16s	remaining: 1m 32s
487:	learn: 0.5910570	total: 8.18s	remaining: 1m 32s
488:	learn: 0.5908975	total: 8.2s	remaining: 1m 32s
489:	learn: 0.5907274	total: 8.22s	remaining: 1m 32s
490:	learn: 0.5905605	total: 8.24s	remaining: 1m 32s
491:	learn: 0.5903970	total: 8.25s	remaining: 1m 32s
492:	learn: 0.5902355	total: 8.27s	remaining: 1m 32s
493:	learn: 0.5900750	total: 8.29s	remaining: 1m 32s
494:	learn: 0.5899184	total: 8.31s	remaining: 1m 32s
495:	learn: 0.5897723	total: 8.33s	remaining: 1m 32s
496:	learn: 0.5895899	total: 8.35s	remaining: 1m 32s
497:	learn: 0.5894226	total: 8.37s	remaining: 1m 32s
498:	learn: 0.5892771	total: 8.38s	remaining: 1m 32s
499:	learn: 0.5891202	total: 8.4s	remaining: 1m 32s
500:	learn: 0.5889485	total: 8.42s	remaining: 1m 32s
501:	learn: 0.5887915	total: 8.44s	remaining: 1m

647:	learn: 0.5674185	total: 10.9s	remaining: 1m 30s
648:	learn: 0.5672757	total: 10.9s	remaining: 1m 30s
649:	learn: 0.5671692	total: 11s	remaining: 1m 30s
650:	learn: 0.5670228	total: 11s	remaining: 1m 30s
651:	learn: 0.5668847	total: 11s	remaining: 1m 30s
652:	learn: 0.5667971	total: 11s	remaining: 1m 29s
653:	learn: 0.5666834	total: 11s	remaining: 1m 29s
654:	learn: 0.5666204	total: 11s	remaining: 1m 29s
655:	learn: 0.5664734	total: 11s	remaining: 1m 29s
656:	learn: 0.5663267	total: 11s	remaining: 1m 29s
657:	learn: 0.5662460	total: 11s	remaining: 1m 29s
658:	learn: 0.5661017	total: 11.1s	remaining: 1m 29s
659:	learn: 0.5659800	total: 11.1s	remaining: 1m 29s
660:	learn: 0.5658479	total: 11.1s	remaining: 1m 29s
661:	learn: 0.5656943	total: 11.1s	remaining: 1m 29s
662:	learn: 0.5655577	total: 11.1s	remaining: 1m 29s
663:	learn: 0.5654462	total: 11.1s	remaining: 1m 29s
664:	learn: 0.5652821	total: 11.1s	remaining: 1m 29s
665:	learn: 0.5651385	total: 11.2s	remaining: 1m 29s
666:	learn:

...

5298:	learn: 0.3368869	total: 1m 47s	remaining: 14.2s
5299:	learn: 0.3368800	total: 1m 47s	remaining: 14.2s
5300:	learn: 0.3368666	total: 1m 47s	remaining: 14.2s
5301:	learn: 0.3368656	total: 1m 47s	remaining: 14.2s
5302:	learn: 0.3368525	total: 1m 47s	remaining: 14.1s
5303:	learn: 0.3368396	total: 1m 47s	remaining: 14.1s
5304:	learn: 0.3368284	total: 1m 47s	remaining: 14.1s
5305:	learn: 0.3368110	total: 1m 47s	remaining: 14.1s
5306:	learn: 0.3367997	total: 1m 47s	remaining: 14.1s
5307:	learn: 0.3367868	total: 1m 47s	remaining: 14s
5308:	learn: 0.3367731	total: 1m 47s	remaining: 14s
5309:	learn: 0.3367641	total: 1m 47s	remaining: 14s
5310:	learn: 0.3367535	total: 1m 47s	remaining: 14s
5311:	learn: 0.3367422	total: 1m 47s	remaining: 14s
5312:	learn: 0.3367243	total: 1m 47s	remaining: 13.9s
5313:	learn: 0.3367158	total: 1m 47s	remaining: 13.9s
5314:	learn: 0.3367020	total: 1m 47s	remaining: 13.9s
5315:	learn: 0.3366884	total: 1m 47s	remaining: 13.9s
5316:	learn: 0.3366711	total: 1m 47s	r

5453:	learn: 0.3349779	total: 1m 51s	remaining: 11.1s
5454:	learn: 0.3349584	total: 1m 51s	remaining: 11.1s
5455:	learn: 0.3349425	total: 1m 51s	remaining: 11.1s
5456:	learn: 0.3349333	total: 1m 51s	remaining: 11.1s
5457:	learn: 0.3349208	total: 1m 51s	remaining: 11.1s
5458:	learn: 0.3349127	total: 1m 51s	remaining: 11s
5459:	learn: 0.3348961	total: 1m 51s	remaining: 11s
5460:	learn: 0.3348851	total: 1m 51s	remaining: 11s
5461:	learn: 0.3348730	total: 1m 51s	remaining: 11s
5462:	learn: 0.3348574	total: 1m 51s	remaining: 11s
5463:	learn: 0.3348497	total: 1m 51s	remaining: 10.9s
5464:	learn: 0.3348404	total: 1m 51s	remaining: 10.9s
5465:	learn: 0.3348390	total: 1m 51s	remaining: 10.9s
5466:	learn: 0.3348288	total: 1m 51s	remaining: 10.9s
5467:	learn: 0.3348113	total: 1m 51s	remaining: 10.8s
5468:	learn: 0.3348036	total: 1m 51s	remaining: 10.8s
5469:	learn: 0.3348003	total: 1m 51s	remaining: 10.8s
5470:	learn: 0.3347907	total: 1m 51s	remaining: 10.8s
5471:	learn: 0.3347844	total: 1m 51s	r

5612:	learn: 0.3330125	total: 1m 54s	remaining: 7.91s
5613:	learn: 0.3330018	total: 1m 54s	remaining: 7.89s
5614:	learn: 0.3329867	total: 1m 54s	remaining: 7.87s
5615:	learn: 0.3329684	total: 1m 54s	remaining: 7.85s
5616:	learn: 0.3329507	total: 1m 54s	remaining: 7.83s
5617:	learn: 0.3329412	total: 1m 54s	remaining: 7.8s
5618:	learn: 0.3329276	total: 1m 54s	remaining: 7.79s
5619:	learn: 0.3329156	total: 1m 54s	remaining: 7.76s
5620:	learn: 0.3329017	total: 1m 54s	remaining: 7.74s
5621:	learn: 0.3328923	total: 1m 54s	remaining: 7.72s
5622:	learn: 0.3328821	total: 1m 54s	remaining: 7.7s
5623:	learn: 0.3328695	total: 1m 54s	remaining: 7.68s
5624:	learn: 0.3328636	total: 1m 54s	remaining: 7.66s
5625:	learn: 0.3328562	total: 1m 54s	remaining: 7.64s
5626:	learn: 0.3328462	total: 1m 54s	remaining: 7.62s
5627:	learn: 0.3328269	total: 1m 55s	remaining: 7.6s
5628:	learn: 0.3328168	total: 1m 55s	remaining: 7.58s
5629:	learn: 0.3328019	total: 1m 55s	remaining: 7.56s
5630:	learn: 0.3327859	total: 1

5769:	learn: 0.3311407	total: 1m 59s	remaining: 4.74s
5770:	learn: 0.3311247	total: 1m 59s	remaining: 4.72s
5771:	learn: 0.3311133	total: 1m 59s	remaining: 4.7s
5772:	learn: 0.3311034	total: 1m 59s	remaining: 4.68s
5773:	learn: 0.3310914	total: 1m 59s	remaining: 4.66s
5774:	learn: 0.3310804	total: 1m 59s	remaining: 4.64s
5775:	learn: 0.3310724	total: 1m 59s	remaining: 4.62s
5776:	learn: 0.3310619	total: 1m 59s	remaining: 4.6s
5777:	learn: 0.3310512	total: 1m 59s	remaining: 4.58s
5778:	learn: 0.3310412	total: 1m 59s	remaining: 4.56s
5779:	learn: 0.3310278	total: 1m 59s	remaining: 4.54s
5780:	learn: 0.3310105	total: 1m 59s	remaining: 4.52s
5781:	learn: 0.3310007	total: 1m 59s	remaining: 4.5s
5782:	learn: 0.3309890	total: 1m 59s	remaining: 4.48s
5783:	learn: 0.3309767	total: 1m 59s	remaining: 4.46s
5784:	learn: 0.3309688	total: 1m 59s	remaining: 4.44s
5785:	learn: 0.3309601	total: 1m 59s	remaining: 4.42s
5786:	learn: 0.3309501	total: 1m 59s	remaining: 4.39s
5787:	learn: 0.3309341	total: 1

5934:	learn: 0.3292330	total: 2m 2s	remaining: 1.35s
5935:	learn: 0.3292138	total: 2m 2s	remaining: 1.32s
5936:	learn: 0.3291931	total: 2m 2s	remaining: 1.3s
5937:	learn: 0.3291861	total: 2m 2s	remaining: 1.28s
5938:	learn: 0.3291765	total: 2m 3s	remaining: 1.26s
5939:	learn: 0.3291688	total: 2m 3s	remaining: 1.24s
5940:	learn: 0.3291553	total: 2m 3s	remaining: 1.22s
5941:	learn: 0.3291420	total: 2m 3s	remaining: 1.2s
5942:	learn: 0.3291259	total: 2m 3s	remaining: 1.18s
5943:	learn: 0.3291150	total: 2m 3s	remaining: 1.16s
5944:	learn: 0.3291097	total: 2m 3s	remaining: 1.14s
5945:	learn: 0.3290958	total: 2m 3s	remaining: 1.12s
5946:	learn: 0.3290830	total: 2m 3s	remaining: 1.1s
5947:	learn: 0.3290745	total: 2m 3s	remaining: 1.08s
5948:	learn: 0.3290686	total: 2m 3s	remaining: 1.06s
5949:	learn: 0.3290557	total: 2m 3s	remaining: 1.03s
5950:	learn: 0.3290409	total: 2m 3s	remaining: 1.01s
5951:	learn: 0.3290248	total: 2m 3s	remaining: 995ms
5952:	learn: 0.3290149	total: 2m 3s	remaining: 97

In [226]:
from sklearn.metrics import roc_auc_score, f1_score

# Get the predicted probabilities of the positive class
y_pred_prob_ensemble = ensemble.predict_proba(X_test)[:, 1]

# Compute the AUC score of the ensemble model
auc_ensemble = roc_auc_score(y_test, y_pred_prob_ensemble)
print('Ensemble AUC:{:.1f}%'.format(auc_ensemble*100))

# Compute the F1 score of the ensemble model
f1_ensemble = f1_score(y_test, y_pred_ensemble)
print('Ensemble F1 Score: {:.1f}%'.format(f1_ensemble*100))

Ensemble AUC:84.9%
Ensemble F1 Score: 63.4%


In [228]:
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred_ensemble))

Confusion Matrix:
 [[894 141]
 [135 239]]
